In [0]:
# defing the stage table, which pulls the data from the orders_stage table
stage_table = 'autoloader_catalog.orders_schema.orders_stage'

In [0]:
# Reading the data from order stage table
df = spark.read.table(stage_table)


#### creating the orders_final table
CREATE TABLE IF NOT EXISTS dbx_catalog.orders_schema.orders_final(
order_num INT,
tracking_num STRING,
pck_recieved_date DATE,
package_deliver_date DATE,
status STRING,
address STRING,
last_update_timestamp TIMESTAMP,
_rescued_data STRING
)

#### drop the table orders_final
drop table dbx_catalog.orders_schema.orders_final

In [0]:
# to enable sql operations on the dataframe
df.createOrReplaceTempView('final_df')

In [0]:
%sql
MERGE INTO autoloader_catalog.orders_schema.orders_final AS target
USING final_df AS stage
ON stage.tracking_num = target.tracking_num -- if the condition matched, record will delete
WHEN MATCHED THEN DELETE;

INSERT INTO autoloader_catalog.orders_schema.orders_final  -- adding the updated and the new records 
SELECT * FROM final_df;


if not spark._jsparkSession.catalog().tableExists(final_table):
    df.write.format('delta').saveAsTable(final_table)

else:
    target_table = DeltaTable.forName(spark, final_table)
    
    merge_condition ="stage.tracking_num = target.tracking_num"

    target_table.alias("target").merge(
        df.alias("stage"),
        merge_condition
    ).whenMatchedDelete().execute()
    
    df.write.format('delta').mode('append').saveAsTable(final_table)